![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## plants-watering-auto

This is a short notebook to collect and display data from "Phidgets" sensors around a plant and allow the user to automatically water the plant on a daily basis.

We have tried to make this notebook as friendly as possible for beginner programmers and hardware hackers.

Ask M. Lamoureux or Mary Grant (Callysto ambassadors) for details. August, 2023.

<h2 style="color: red">IMPORTANT</h2>
You MUST click the <span style="color:red">BIG, RED DISCONNECT BUTTON</span> at the end of this notebook when you are done. This tells the software to release the Phidget device, so it can be used by other notebooks you might try later.

<h2 style="color: red">WARNING</h2>
Water can cause all kinds of damage. An automatic watering system can lead to accidental spills that could damage your computer, your furniture and more. Be sure to set up your plant watering system someplace safe, where a water spill will not cause trouble.

One safe way is to place the plant and watering system in a waterproof plastic container. 

## Overview

The purpose of this notebook is to collect data from certain hardware sensors called [Phidgets](https://www.phidgets.com/education/learn/projects/plant-kit/). This kit has all the sensors we need to monitor the environment of the plant, along with a water pump and control. The data can be viewed directly on the computer screen, and the water pump can be activated to water the plant.

We recommend you follow the detailed setup for the Plant Kit provided by the Phidget company. The instructions are here: https://www.phidgets.com/education/learn/projects/plant-kit/introduction/

The [Phidget Plant Kit](https://www.phidgets.com/education/learn/projects/plant-kit/) contains several components. To set up this notebook, make sure you have the following:

- the Phidget VINT Hub (HUB0000), with cables attached
- the DC power supply (PSU2000), attached to Port 1 on the VINT Hub
- the water pump (KIT4014), attached to the power supply, 
- the moisture Phidget (device type HUM1100), attached to the VINT Hub
- the light sensor (device type LUX1000_0), attached to the VINT Hub
- and optionally, the temperature/humidity sensor (device type HUM1001_01).

There are four main steps in this notebook.

1. Set up gauges and buttons on the screen to view the sensor data and control the water pumo.
2. Set up the sensors and pump as software inputs/outputs to computer and connet to the software can communicate.
3. Test that the water pump works by pressing the on-screen button, and turn on the auto-watering system.
5. Observe the system over several days, to adjust the moisture level and check the computer is watering automatically when it should. 

Run the cells in this notebook, one cell at a time. This will give you the chance to respond to any errors and fix things if necessary. You may also select "Run All" to execute the whole notebook -- just be prepared to respond to any error messages. 

You should have a plant, with the sensors, water reservoir, and pump  nearby. This what the plant setup should look like:

<div align="center">
<img src="images/plant-water.jpg" alt="A plant with sensors and water pump" width="400"/><br>
A plant with sensors and water pump.
</div>

Here is some more detail on the various devices we are using. 
<div align="center">
<img src="images/sensors.jpg" alt="Details of the pump and sensors" width="600"/><br>
1: Pump. 2: DC power supply. 3: Moisture probe. 4: Light sensor. 5: Humidity sensor. 6: VINT Hub
</div>

The notebook runs both Python code and Javascript (JS) code. It would be nice to do everything in Python, but it seems Javascript is necessary to communicate with the Phidgets in a Jupyter notebook. Fortunately, the Javascript code here is easy enough to read and you will not need to change it. 

## Step 1. Setting up the user interface

We first set up some gauges to display values for temperature, humidity, soil moisture and light levels. This uses the Plotly library, which is loaded in with the **import** command in Python. Other libraries are also loaded here.  

We include two buttons, one to start/stop the automatic watering, and the other to force immediate watering of the plant. 

There is a slider, to adjust the moisture setpoint. The plant will only be watered if the moisture sensor goes below this level. 

Finally, there is a text box to record when the plant was watered. 

In [ ]:
from IPython.display import display, Javascript
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
from datetime import datetime
from time import sleep
import threading

# the four sensor gauges
g_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [10, 40]}}
)

g_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

g_moist = go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Moisture"},
    gauge = {'axis': {'range': [0, 1.0]}}
)

g_light = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Light Level"},
    gauge = {'axis': {'range': [0, 10000]}}
)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],[{'type' : 'domain'}, {'type' : 'domain'}]],
    vertical_spacing = 0.35
)
fig.append_trace(g_temp, row=1, col=1)
fig.append_trace(g_hum, row=1, col=2)
fig.append_trace(g_moist, row=2, col=1)
fig.append_trace(g_light, row=2, col=2)

gauges = go.FigureWidget(fig)

## The moisture trigger setpoint

moist_slider = widgets.FloatSlider(
    value=.3,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Moisture setpoint:',
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='400px', height='80px'),
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

# the data update button
data_button = widgets.Button(
    description='Update gauges',
    tooltip='Update data to gauges',
    disabled=False,
    button_style='success',
)

# the watering button
water_button = widgets.Button(
    description='Click to water',
    tooltip='Water for two seconds',
    disabled=False,
    button_style='success',
)

# the auto-watering Start/Stop button
auto_button = widgets.Button(
    description='Start Auto Watering',
    tooltip='Start or stop the auto-watering',
    disabled=False,
    button_style='success',
)

record_text = widgets.Textarea(
    value='Record of the watering',
    description='Last watered:',
    disabled=False,
    layout=widgets.Layout(width='40%', height='80px')
)

dashboard = widgets.VBox([gauges,moist_slider,widgets.HBox([data_button,water_button,auto_button]),record_text])
print('dashboard ready')

## Viewing the user interface

We can view the user interface with the display command. However, the gauges and buttons are not live yet, so it doesn't do much yet. 

In [ ]:
display(dashboard)

## Step 2: Connecting the sensor devices

At this point, you should connect the Phidgets hardware to your computer. This includes six separate devices:
- the Phidget VINT device, attached to the computer's USB port
- the DC power supply, attached to Port 1 on the VINT device
- The water pump, attached to the DC power supply
- the soil moisture sensor, attached to the VINT
- the light sensor, attached to the VINT
- optionally, the combined temperature/humidity sensor, attached to the VINT

When you first attach the VINT device to the USB port, your computer may ask you whether you wish to connect to this USB device. Please answer "yes" to this security request. 

We now go set up the software to communicate with the Phidget devices.

### Load the phidget library for Javascript

In [ ]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

In [ ]:
## We pause for a second here, to allow some time for the library to load in the background
sleep(1)

### Opening the USB connection

We now open a connection between the computer and your Phidget VINT device. The VINT must be plugged into your computer's USB port. Run the following cell, and follow the prompts to select the VINT device (a list appears that you should click on). This will pair the device with your computer. 

In [ ]:
%%js
if (window.usbconn === undefined) {
    element.text("Creating a new USB Connection.");
    window.usbconn = new phidget22.USBConnection();    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

In [ ]:
## We rest for a bit while the USB connects
sleep(1)

### Confirm the USB connection 

Run the following code to see if the device is connected. It should say "true." If it does not, check your cable connections. You may also need to check the security settings on your computer to allow new USB devices to get connected. 

In [ ]:
%%js
element.text("Is the USB device connected? " + (usbconn.connected ? "YES.":"NO."));

### Connect the sensors

We make a request to open the four different sensors, for temperature, humidity, soil moisture and light level, as well as the control for the water pump. Be sure your sensors are plugged into the VINT device, and the pump is on Port number one. 

If you are missing a sensor or two, that is okay. The data collection for the other sensors will still work. The cells below will connect the sensors, then check to see that they are attached. 

In [ ]:
%%js
window.tempSensor = new phidget22.TemperatureSensor();
window.humSensor = new phidget22.HumiditySensor();
window.moistSensor = new phidget22.VoltageRatioInput();
window.liteSensor = new phidget22.LightSensor();
window.pump = new phidget22.DigitalOutput();
pump.setHubPort(1);
pump.setIsHubPortDevice(true);

async function setup_sensors() {
    let errorCode = 0;
    try {await tempSensor.open(1000);} catch {errorCode |= 1;}    
    try {await humSensor.open(1000);} catch {errorCode |= 2;}
    try {await moistSensor.open(1000);} catch {errorCode |= 4;}
    try {await liteSensor.open(1000);} catch {errorCode |= 8;} 
    try {await pump.open(1000);} catch {errorCode |= 16;}
    return errorCode
}

setup_sensors()

In [ ]:
## We rest for ten seconds while the Phidgets try to connect.
sleep(10)

### Confirm the sensor connections

Run the following code to see if the five devices have all connected. All five lines should say "YES." If you don't have a sensor connected, this notebook will still work -- you just won't get data from that sensor. 

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + (humSensor.attached ? "YES.":"NO."));
element.append("<br>Is the temperature sensor attached? " + (tempSensor.attached ? "YES.":"NO."));
element.append("<br>Is the moisture sensor attached? " + (moistSensor.attached ? "YES.":"NO."));
element.append("<br>Is the light sensor attached? " + (liteSensor.attached ? "YES.":"NO."))
element.append("<br>Is the pump attached? " + (pump.attached ? "YES.":"NO."));

## Setting up the buttons

Finally, we connect the buttons in the user interface to the sensors.

The following sets up code to make the buttons and sliders functional. 

In [ ]:
## The button/slider user interface

## Some code to update the data
def update_gauges(button):
    display(Javascript("""
        if (tempSensor.attached) {
            IPython.notebook.kernel.execute(
                "gauges.data[0]['value'] = " + tempSensor.temperature); }
        if (humSensor.attached) {
        IPython.notebook.kernel.execute(
            "gauges.data[1]['value'] = " + humSensor.humidity); }
        if (moistSensor.attached) {
        IPython.notebook.kernel.execute(
            "gauges.data[2]['value'] = " + moistSensor.voltageRatio); }
        if (liteSensor.attached) {
        IPython.notebook.kernel.execute(
            "gauges.data[3]['value'] = " + liteSensor.illuminance); }
    """))
data_button.on_click(update_gauges)

# Click the water button to turn on the water for 2 seconds (2000 milliseconds in setTimeout)
# We also record the event in a text box
def water_button_do(widget):
    widget.button_style = 'info'
    widget.description='Watering...'
    display(Javascript("""
        pump.setState(true);
        setTimeout(function() {pump.setState(false);}, 2000);
        """))
    sleep(2)          ## Delay 2 seconds to let setTimeout run above
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    record_text.value = now + ', Moisture = ' + str(gauges.data[2]['value']) + '\n' + record_text.value
    widget.button_style = 'success'
    widget.description='Click to water'
water_button.on_click(water_button_do)

# conditionally water the plant, if the moisture is too low. We also record the event in the text box
def water_if_dry():
    if (gauges.data[2]['value'] < moist_slider.value):
        water_button_do(water_button)

 # a flag to monitor if the auto-watering is on or not. The "event" is thread-safe
is_watering = threading.Event()
is_watering.clear()
       
# The watering loop will try to water each day, for up to 100 days
def watering_loop():
    for i in range(100):
        update_gauges(0)
        sleep(2)
        if (is_watering.is_set()):
            water_if_dry()
            for j in range(60*60*24):  # check every second of a day, to see if we continue auto-watering
                if not is_watering.is_set():
                    return
                sleep(1) # pause a second, then continue to count off seconds in the day
        else:
            break
    
# Click the auto button to start and stop the automatic watering
def auto_button_do(widget):
    global is_watering
    if (widget.description=='Start Auto Watering'):
        widget.description='Stop Auto Watering'
        widget.button_style='danger'
        is_watering.set()
        thread = threading.Thread(target=watering_loop) 
        thread.start()
    else:
        widget.description='Start Auto Watering'
        widget.button_style='success'
        is_watering.clear()
auto_button.on_click(auto_button_do)

print('buttons ready')

## Step 3: View and use the user interface

Now that everything is connected, we can view the user interface below. The gauges should be live, showing the values from the sensors.

Try the  "Update Gauges" button. It should show the latest sensor data on the gauges.

Try the  "Click to Water" button. It should run the water pump for two seconds.

Then try the "Start Auto Watering" button. This will check every day to see how moist the soil is. If the moisture is below the setpoint, as set by the slider, the computer will run water pump for two seconds. 

While Auto Watering is active, the button turns Red, and changes to say "Stop Auto Watering." Click on it if you want to stop the auto-watering. 

In [ ]:
display(dashboard)

## Step 4: Observing the results

You'll need to be patient to see the results, as the code only checks the watering once a day. 

You can adjust the moisture setpoint at any time, using the slider. If you want to test it, just stop and re-start the auto-watering using the Auto Watering button. At start-up, the code always tests to see if the moisture gauge is reading below the set point. If it is below, then the pump gives a squirt of water.

Try running this for several days. Does the system keep your plant watered?

## Final Step. Closing down the sensors

Once you are all done with the automatic watering, it is **really important** to close the sensors now. Otherwise, the next next person to use the computer may not be able to connect to the Phidget devices. If you just quit the notebook, the sensors may not disconnect properly, again which can cause trouble the next time someone tries to connect to the Phidgets.

So, don't skip this next step.

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook. 

In [ ]:
def doDisconnect(b):
    is_watering.clear()
    display(Javascript("""
        (async () => {
            await humSensor.close();
            await tempSensor.close();
            await moistSensor.close();
            await liteSensor.close();
            await pump.close();
            usbconn.close();
            usbconn.delete();
            delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
        })();
    """))

run_button = widgets.Button(
    description = 'IMPORTANT: Click to disconnect', 
        button_style='danger',layout=widgets.Layout(width='50%', height='80px')
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)


### Confirm

You can confirm the Phidgets are open or closed by running the following cell. 

If any device is still attached (true), try clicking the button above, again.

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + (humSensor.attached ? "YES.":"NO."));
element.append("<br>Is the temperature sensor attached? " + (tempSensor.attached ? "YES.":"NO."));
element.append("<br>Is the moisture sensor attached? " + (moistSensor.attached ? "YES.":"NO."));
element.append("<br>Is the light sensor attached? " + (liteSensor.attached ? "YES.":"NO."))
element.append("<br>Is the pump attached? " + (pump.attached ? "YES.":"NO."));

## Conclusion

We have shown how to display Phidget sensor data from a plant and how to control the water pump for the plant. Using the auto-watering system, the computer will water your plant each day in order to keep the moisture above the given setpoint. 

You might like to go back to the data collection notebooks: 
- [plants-phidgets-data-google.ipynb](plants-phidgets-data-google.ipynb)
- [plants-phidgets-data-ethercalc.ipynb](plants-phidgets-data-ethercalc.ipynb)

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)